<a href="https://colab.research.google.com/github/DayalStrub/ecir2021tutorial/blob/main/other/2-simplest-thing-WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q python-terrier
!pip install -q git+https://github.com/terrier-org/pyterrier.git

In [2]:
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

In [4]:
next(iter(dataset.get_corpus_iter()))

OrderedDict([('title',
              'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia'),
             ('doi', '10.1186/1471-2334-1-6'),
             ('date', '2001-07-04'),
             ('abstract',
              'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common

In [5]:
N = 10_000
pt_index_path = './index_cord19'

# create the index, using the IterDictIndexer indexer 
indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True) # , blocks=True

# we give the dataset get_corpus_iter() directly to the indexer
# while specifying the fields to index and the metadata to record
index_ref = indexer.index(
    (datum for i, datum in enumerate(dataset.get_corpus_iter()) if i < N),
    fields=('abstract',), # TODO should this be a list?
    # meta={'docno' : 26, 'text' : 2048, 'abstract' : 2048}
    meta=['docno','abstract'],
    )

index = pt.IndexFactory.of(index_ref)

08:14:06.606 [ForkJoinPool-1-worker-3] WARN  o.t.structures.i
ndexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed

08:14:20.714 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 2143 empty documents


In [6]:
print(index.getCollectionStatistics().toString())

Number of documents: 10000
Number of terms: 36191
Number of postings: 673758
Number of fields: 1
Number of tokens: 1040094
Field names: [abstract]
Positions:   false



In [7]:
queries = dataset.get_topics(variant='title').head(1)
queries.head()

,qid,query
0,1,coronavirus origin


In [8]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
rm3 = pt.rewrite.RM3(index)
bm25_text = pt.BatchRetrieve(index, wmodel="BM25", metadata=["docno", "abstract"])

rm3_pipe = bm25 >> rm3 >> bm25_text

In [9]:
# TODO Question: in rm3_pipe, is final bm25 just reranking initial bm25 results, or does it re-consider all docs?

In [10]:
# TODO figure out how to implement PR Feedback using logistic regression on bag of words a la Lin

In [11]:
df_out = rm3_pipe.transform(queries)
df_out.loc[df_out["qid"] == "1", :]

,qid,docid,docno,abstract,rank,score,query_0,query
0,1,2257,8y24j34j,Middle East respiratory syndrome coronavirus i...,0,19.049835,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
1,1,4582,2jq626ye,A novel coronavirus (2019-nCoV) originating in...,1,13.717366,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
2,1,1394,vnafx1ng,The phosphoprotein (P) gene of most Paramyxovi...,2,11.114468,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
3,1,1616,hp5x637c,"BACKGROUND: In 2007, a novel bunyavirus was fo...",3,8.417806,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
4,1,4936,41ui4lqc,Emerging viruses represent a continuous threat...,4,8.340423,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
...,...,...,...,...,...,...,...,...
995,1,1727,5d64esp4,"RNA viruses exhibit substantial structural, ec...",995,0.986444,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
996,1,1685,bkreu1s2,"Collectively, viruses have the greatest geneti...",996,0.986241,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
997,1,7870,o2cijp5a,In the present report we provide the strain di...,997,0.986241,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
998,1,4489,78ur1j2p,Despite all the research done on the first Zik...,998,0.983408,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...


In [12]:
# NOTE indeed/actually a function
pt.Experiment(
    retr_systems=[bm25, rm3_pipe],
    names=["BM25", "BM25-RM3-BM25",],
    topics=queries,
    qrels=dataset.get_qrels(), # TODO figure out cord19 qrels
    eval_metrics=["map"]
              )

,name,map
0,BM25,0.000020
1,BM25-RM3-BM25,0.000018
